---
title: "Skrub sprint"
title-block-banner: true
date: 2025-10-29
subtitle: "Women in Machine Learning & Data Science"
author: "Riccardo Cappuzzo"
institute: "Probabl"
format: 
    revealjs:
        slide-number: c/t
        show-slide-number: all
        preview-links: auto
        embed-resources: false
        transition: slide
        theme: simple
        logo: images/skrub.svg
        css: style.css
        footer: "https://skrub-data.org/skrub-materials/"
incremental: false
params: 
    version: "base"
---



## `whoami`  

::: {.incremental}

- I am a research engineer at Inria as part of the P16 project, and I am the lead developer of skrub ![](images/inria.png){width=250}

- I'm Italian, but I don't drink coffee, wine, and I like pizza with fries ![](images/raora.png){width=50}

- I did my PhD in Côte d'Azur, but I moved to Paris because it was too sunny and 
I don't like the sea ![](images/nice.jpg){width=250}

:::

## Who are you? 

::: {.incremental}

- Who has already made contributions in open source? 

- Who has heard of skrub before today?
:::

## Roadmap for the presentation

- What is skrub
- Contributing to skrub: subjects
- Contributing to skrub: setting up the environment

QR code for the presentation at the end! 

# What is skrub? 

::: {.callout-tip}
Skrub is a Python library that sits between data stored in dataframes and machine
learning with scikit-learn. 
:::

Skrub eases machine learning with dataframes. 

## {auto-animate="true" } 
![](images/powerpuff_girls_1.png)

## {auto-animate="true" } 
![](images/powerpuff_girls_2.png)

## Skrub compatibility
- Skrub is mostly written in Python, but it inlcudes some Javascript
- Skrub is fully compatible with pandas and polars
    - Any feature needs to be supported by both libraries
- Skrub transformers are fully compatible with scikit-learn
    - Transformers need to satisfy some requirements

## First, an example pipeline
1. Gather some data
2. Explore the data
3. Preprocess the data 
4. Perform feature engineering 
5. Build a scikit-learn pipeline
6. ???
7. Profit?  

##  
![](images/here-we-go-again.png)

## `skrub.TableReport` {.smaller auto-animate="true"}

```{.python}
from skrub import TableReport
TableReport(employee_salaries)
```
[TableReport Preview](https://skrub-data.org/skrub-reports/examples/employee_salaries.html){preview-link="true"}


::: {.fragment}
Main features:

- Obtain high-level statistics about the data
- Explore the distribution of values and find outliers
- Discover highly correlated columns 
- Export and share the report as an `html` file
:::


## `skrub.TableReport` {auto-animate="true"}

- The report uses uses Jinjia templates and Javascript for interactivity
- The backend is in Python
- Space is limited, need to maximize information density. 
- Light > feature-rich (no plotly)

## Data cleaning with pandas/polars: setup {.smaller auto-animate="true"}

::: {.panel-tabset}

### Pandas

In [ ]:
# | echo: true
import pandas as pd
import numpy as np

data = {
    "Int": [2, 3, 2],  # Multiple unique values
    "Const str": ["x", "x", "x"],  # Single unique value
    "Str": ["foo", "bar", "baz"],  # Multiple unique values
    "All nan": [np.nan, np.nan, np.nan],  # All missing values
    "All empty": ["", "", ""],  # All empty strings
    "Date": ["01 Jan 2023", "02 Jan 2023", "03 Jan 2023"],
}

df_pd = pd.DataFrame(data)
display(df_pd)

### Polars

In [ ]:
#| echo: true
import polars as pl
import numpy as np
data = {
    "Int": [2, 3, 2],  # Multiple unique values
    "Const str": ["x", "x", "x"],  # Single unique value
    "Str": ["foo", "bar", "baz"],  # Multiple unique values
    "All nan": [np.nan, np.nan, np.nan],  # All missing values
    "All empty": ["", "", ""],  # All empty strings
    "Date": ["01 Jan 2023", "02 Jan 2023", "03 Jan 2023"],
}

df_pl = pl.DataFrame(data)
display(df_pl)

:::


## Nulls, datetimes, constant columns with pandas/polars {.smaller auto-animate="true"}

:::{.panel-tabset}
### Pandas

In [ ]:
#| echo: true
# Parse the datetime strings with a specific format
df_pd['Date'] = pd.to_datetime(df_pd['Date'], format='%d %b %Y')

# Drop columns with only a single unique value
df_pd_cleaned = df_pd.loc[:, df_pd.nunique(dropna=True) > 1]

# Function to drop columns with only missing values or empty strings
def drop_empty_columns(df):
    # Drop columns with only missing values
    df_cleaned = df.dropna(axis=1, how='all')
    # Drop columns with only empty strings
    empty_string_cols = df_cleaned.columns[df_cleaned.eq('').all()]
    df_cleaned = df_cleaned.drop(columns=empty_string_cols)
    return df_cleaned

# Apply the function to the DataFrame
df_pd_cleaned = drop_empty_columns(df_pd_cleaned)

### Polars

In [ ]:
#| echo: true 
# Parse the datetime strings with a specific format
df_pl = df_pl.with_columns([
    pl.col("Date").str.strptime(pl.Date, "%d %b %Y", strict=False).alias("Date")
])

# Drop columns with only a single unique value
df_pl_cleaned = df_pl.select([
    col for col in df_pl.columns if df_pl[col].n_unique() > 1
])

# Import selectors for dtype selection
import polars.selectors as cs

# Drop columns with only missing values or only empty strings
def drop_empty_columns(df):
    all_nan = df.select(
        [
            col for col in df.select(cs.numeric()).columns if 
            df [col].is_nan().all()
        ]
    ).columns
    
    all_empty = df.select(
        [
            col for col in df.select(cs.string()).columns if 
            (df[col].str.strip_chars().str.len_chars()==0).all()
        ]
    ).columns

    to_drop = all_nan + all_empty

    return df.drop(to_drop)

df_pl_cleaned = drop_empty_columns(df_pl_cleaned)

:::

## `skrub.Cleaner` {.smaller auto-animate="true"}

:::{.panel-tabset}
### Pandas

In [ ]:
#| echo: true
from skrub import Cleaner
cleaner = Cleaner(drop_if_constant=True, datetime_format='%d %b %Y')
df_cleaned = cleaner.fit_transform(df_pd)
display(df_cleaned)

### Polars

In [ ]:
#| echo: true
from skrub import Cleaner
cleaner = Cleaner(drop_if_constant=True, datetime_format='%d %b %Y')
df_cleaned = cleaner.fit_transform(df_pl)
display(df_cleaned)

:::

## `skrub.Cleaner` {auto-animate="true"}
- The actual transformations are performed in part by `skrub.DropUninformative`.
- New criteria for selecting columns should go in `DropUninformative`. 


## `skrub.DatetimeEncoder` {auto-animate="true" visibility="uncounted" .smaller}

In [ ]:
import polars as pl
data = {
    'date': ['2023-01-01 12:34:56', '2023-02-15 08:45:23', '2023-03-20 18:12:45'],
    'value': [10, 20, 30]
}
df = pl.DataFrame(data)

In [ ]:
#| echo: true
from skrub import DatetimeEncoder, ToDatetime

X_date = ToDatetime().fit_transform(df["date"])
de = DatetimeEncoder(resolution="second")
# de = DatetimeEncoder(periodic_encoding="spline")
X_enc = de.fit_transform(X_date)
print(X_enc)

## Encoding categorical (string/text) features
Categorical features have a "**cardinality**": the number of unique values

::: {.incremental}

- Low cardinality: `OneHotEncoder`
- High cardinality (>40 unique values): `skrub.StringEncoder`
- Text: `skrub.TextEncoder` and pretrained models from HuggingFace Hub

:::

## Encoding _all the features_: `TableVectorizer` { auto-animate="true"}


In [ ]:
#| echo: true
from skrub import TableVectorizer

table_vec = TableVectorizer()
df_encoded = table_vec.fit_transform(df)

::: {.fragment}
- Apply the `Cleaner` to all columns
- Split columns by dtype and # of unique values
- Encode each column separately
:::


## Encoding _all the features_: `TableVectorizer` {.smaller auto-animate="true"}

![](images/skrub-table-vectorizer.png)


## Build a predictive pipeline with `tabular_pipeline` {auto-animate="true" .smaller}
```{.python}
import skrub
from sklearn.linear_model import Ridge
model = skrub.tabular_pipeline(Ridge())
```

![](images/skrub-tabular-pipeline-linear-model.png){fig-align="center"}

# Advanced skrub: Data Ops

## DataOps...

::: {.incremental}
- Extend the `scikit-learn` machinery to complex multi-table operations, and take care of data leakage
- Track all operations with a computational graph (a *Data Ops plan*)
- Are transparent and give direct access to the underlying object
- Allow tuning any operation in the Data Ops plan
- Guarantee that all operations are reproducible
- Can be persisted and shared easily 
:::

## How do DataOps work, though?  {.smaller}
DataOps **wrap** around *user operations*, where user operations are:

- any dataframe operation (e.g., merge, group by, aggregate etc.)
- scikit-learn estimators (a Random Forest, RidgeCV etc.)
- custom user code (load data from a path, fetch from an URL etc.)

::: {.fragment}

::: {.callout-important}
DataOps _record_ user operations, so that they can later be _replayed_ in the same
order and with the same arguments on unseen data. 
:::
::: 

## Starting with the `DataOps` {auto-animate="true"} 


In [ ]:
#| echo: true
import skrub
data = skrub.datasets.fetch_credit_fraud()

baskets = skrub.var("baskets", data.baskets)
products = skrub.var("products", data.products) # add a new variable

X = baskets[["ID"]].skb.mark_as_X()
y = baskets["fraud_flag"].skb.mark_as_y()

- `baskets` and `products` represent inputs to the pipeline.
- Skrub tracks `X` and `y` so that training and test splits are never mixed. 

## Applying a transformer {auto-animate="true"}

In [ ]:
# | echo: true
from skrub import selectors as s

vectorizer = skrub.TableVectorizer(
    high_cardinality=skrub.StringEncoder()
)
vectorized_products = products.skb.apply(
    vectorizer, cols=s.all() - "basket_ID"
)

##  Executing dataframe operations {auto-animate="true"}

In [ ]:
#| echo: true
aggregated_products = vectorized_products.groupby(
    "basket_ID"
).agg("mean").reset_index()

features = X.merge(
    aggregated_products, left_on="ID", right_on="basket_ID"
)
features = features.drop(columns=["ID", "basket_ID"])

##  Applying a ML model {auto-animate="true"}

In [ ]:
#| echo: true
from sklearn.ensemble import ExtraTreesClassifier  
predictions = features.skb.apply(
    ExtraTreesClassifier(n_jobs=-1), y=y
)

## Inspecting the Data Ops plan
```{.python}
predictions.skb.full_report()
```
<br/>

<a href="dataop_report/index.html" target="_blank">Execution report</a>

Each node:

- Shows a preview of the data resulting from the operation
- Reports the location in the code where the code is defined
- Shows the run time of the node 

## Exporting the plan in a `learner` {.smaller}
The **Learner** is a stand-alone object that works like
a scikit-learn estimator that takes a dictionary as input rather
than just `X` and `y`. 


::: {.fragment}


In [ ]:
#| echo: true
learner = predictions.skb.make_learner(fitted=True)

:::


::: {.fragment}
Then, the `learner` can be pickled ...


In [ ]:
import pickle 

learner_bytes = pickle.dumps(learner)

```{.python}
import pickle

with open("learner.bin", "wb") as fp:
    pickle.dump(learner, fp)
```

In [ ]:
loaded_learner = pickle.loads(learner_bytes)

:::

::: {.fragment}
... loaded and applied to new data:

```{.python}
with open("learner.bin", "rb") as fp:
    loaded_learner = pickle.load(fp)
data = skrub.datasets.fetch_credit_fraud(split="test")
new_baskets = data.baskets
new_products = data.products
loaded_learner.predict({"baskets": new_baskets, "products": new_products})
```

In [ ]:
data = skrub.datasets.fetch_credit_fraud(split="test")
new_baskets = data.baskets
new_products = data.products
loaded_learner.predict(
    {"baskets": new_baskets, "products": new_products}
)

:::


## Hyperparameter tuning in a Data Ops plan 

- `choose_from`: select from the given list of options
- `choose_int`: select an integer within a range
- `choose_float`: select a float within a range
- `choose_bool`: select a bool 
- `optional`: chooses whether to execute the given operation


## Tuning in `scikit-learn` can be complex {.smaller auto-animate="true"}

```{.python}
pipe = Pipeline([("dim_reduction", PCA()), ("regressor", Ridge())])
grid = [
    {
        "dim_reduction": [PCA()],
        "dim_reduction__n_components": [10, 20, 30],
        "regressor": [Ridge()],
        "regressor__alpha": loguniform(0.1, 10.0),
    },
    {
        "dim_reduction": [SelectKBest()],
        "dim_reduction__k": [10, 20, 30],
        "regressor": [Ridge()],
        "regressor__alpha": loguniform(0.1, 10.0),
    },
    {
        "dim_reduction": [PCA()],
        "dim_reduction__n_components": [10, 20, 30],
        "regressor": [RandomForestRegressor()],
        "regressor__n_estimators": loguniform(20, 200),
    },
    {
        "dim_reduction": [SelectKBest()],
        "dim_reduction__k": [10, 20, 30],
        "regressor": [RandomForestRegressor()],
        "regressor__n_estimators": loguniform(20, 200),
    },
]
```
## Tuning with Data Ops is simple! {.smaller} 

```python 
dim_reduction = X.skb.apply(
    skrub.choose_from(
        {
            "PCA": PCA(n_components=skrub.choose_int(10, 30)),
            "SelectKBest": SelectKBest(k=skrub.choose_int(10, 30))
        }, name="dim_reduction"
    )
)
regressor = dim_reduction.skb.apply(
    skrub.choose_from(
        {
            "Ridge": Ridge(alpha=skrub.choose_float(0.1, 10.0, log=True)),
            "RandomForest": RandomForestRegressor(
                n_estimators=skrub.choose_int(20, 200, log=True)
            )
        }, name="regressor"
    )
)
```

## Exploring the hyperparameters {auto-animate="true" .smaller} 

```{.python}
search = pred.skb.get_randomized_search(fitted=True)
search.plot_parallel_coord()
```

In [ ]:
from plotly.io import read_json

fig = read_json("parallel_coordinates_hgbr.json")
fig.update_layout(margin=dict(l=200))

# Resources and contacts

## Examples and guides
- Skrub [example gallery](https://skrub-data.org/stable/auto_examples/data_ops/index.html)
- Skrub [user guide](https://skrub-data.org/stable/documentation.html)
- [Tutorial](https://github.com/skrub-data/EuroSciPy2025) on timeseries 
forecasting at Euroscipy 2025
- [Kaggle notebook](https://www.kaggle.com/code/ryye107/titanic-challenge-with-the-skrub-data-ops) 
on the Titanic survival challenge

##  Getting involved {.smaller}
::: {.nonincremental}
Do you want to learn more? 

- [Skrub website](https://skrub-data.org/stable/) 
- [Skrub materials website](https://skrub-data.org/skrub-materials/index.html)
- [Discord server](https://discord.gg/ABaPnm7fDC)

Follow skrub on:

- [Bluesky](https://bsky.app/profile/skrub-data.bsky.social)
- [LinkedIn](https://www.linkedin.com/company/skrub-data/)

Star skrub on GitHub, or contribute directly: 

- [GitHub repository](https://github.com/skrub-data/skrub/)
:::

# Contributing to skrub: open issues
**[GitHub project](https://github.com/orgs/skrub-data/projects/9/)**

## Before you start working on an issue

::: {.callout-important}
Write a comment on the issue so we know you're working on it.
:::


We want to avoid having multiple people working on the same issue in separate PRs. 

## Legend:

- 😴 : easy issue
- 🍁 : some complexity
- 👺 : hard problem
- 🏃‍♀️ : quick to solve
- 🛌 : likely will take a while
- 🐈 : docs, need to deal with Sphinx

## `TableReport` {.smaller}
- [#1175](https://github.com/skrub-data/skrub/issues/1175) - Better control over
the `TableReport`'s progress display. 👺 🛌
- [#1523](https://github.com/skrub-data/skrub/issues/1523) - Fix the behavior of 
the `TableReport` when `max_plot_columns` is set to `None`. 🍁 🏃
- [#1178](https://github.com/skrub-data/skrub/issues/1178) - Shuffle the rows of 
the `TableReport` example in the home page.  😴 🏃

## New transformers  {.smaller}
- [#1001](https://github.com/skrub-data/skrub/issues/1001) - Add a `DropSimilar`
transformer. 👺 🛌 
- [#710](https://github.com/skrub-data/skrub/issues/710) - Add holidays as features. 👺 🛌
- [#1542](https://github.com/skrub-data/skrub/issues/1542) - Add a transformer 
that parses string columns that include units (kg, $ etc). 👺 🛌
- [#1430](https://github.com/skrub-data/skrub/issues/1430) - Extend `ToDatetime` 
so that it can take a list of datetime formats. 🍁 🛌

## Bugfixes and maintenance {.smaller}
- [#1675](https://github.com/skrub-data/skrub/issues/1675) - Improve error message 
when the `TableReport` receives a lazy Polars dataframe.   😴 🏃
- [#1490](https://github.com/skrub-data/skrub/issues/1490) - Cleaner fails when there 
is an empty polars column name. 👺 🏃

## Documentation {.smaller}

- [#1476](https://github.com/skrub-data/skrub/issues/1476) - DOC: add an example 
dedicated to showing the features of the `TableReport`. 🍁 🛌 🐈
- [#991](https://github.com/skrub-data/skrub/issues/991) - Move the dev docs of
the `TableReport` to the main documentation page. 🍁 🐈
- [#1582](https://github.com/skrub-data/skrub/issues/1582) - Reorganize the 
"Development" section in the top bar. 🍁 🛌 🐈
- [#1425](https://github.com/skrub-data/skrub/issues/1425) - Shorten the note on the 
single-column transformer. 😴 🏃
- [#1660](https://github.com/skrub-data/skrub/issues/1660) - Add different doc
versions to the switcher.  🍁 🐈 🏃
- [#1616](https://github.com/skrub-data/skrub/issues/1616) - Change the numbering
of examples. 😴 🐈 🏃

## Examples  {.smaller}
- [#1629](https://github.com/skrub-data/skrub/issues/1629) - Add an example for the 
`DatetimeEncoder`. 👺 🛌 🐈
- [#1234](https://github.com/skrub-data/skrub/issues/1234) - Shuffle the toxicity
dataset in the example. 😴 🏃
- Any example you can come up with! 

# Contributing to skrub: preparation
Instructions are also available in the Installation page of the webiste:
`https://skrub-data.org/stable/install.html`

## Setting up the repository {.smaller}
First off, you need to fork the skrub repository:
`https://github.com/skrub-data/skrub/fork`

Then, clone the repo on your local machine
```sh
git clone https://github.com/<YOUR_USERNAME>/skrub
cd skrub

```
Add the `upstream` remote to pull the latest version of skrub:
```sh
git remote add upstream https://github.com/skrub-data/skrub.git
```
You can check that the remote has been added with `git remote -v`.

## Setting up the environment {.smaller}
Depends on the tools you use! 

From inside the skrub directory you just cloned: 

::: {.panel-tabset}
### venv
- Create the venv (in the current dir): 
```sh
python -m venv dev-skrub
```
- Activate the venv: 
```sh
source dev-skrub/bin/activate
```
- Install skrub and dependencies: 
```sh
pip install -e ".[dev]"
```

### uv 
- Create the venv (in the current dir): 
```sh
uv venv dev-skrub 
```
- Activate the venv:
```sh
source activate 
```
- Install skrub and dev dependencies:
```sh
uv pip install -e ".[dev]"
```

### conda
- Create the conda environment:
```sh
conda create -n dev-skrub

```
- Activate the enviornment
```sh
conda activate dev-skrub
```
- Install skrub and dependencies: 
```sh
pip install -e ".[dev]"
```


### pixi
- Install pixi: `https://pixi.sh/latest/installation/`
- Go to the skrub folder
- Install an environment:
```sh
pixi install dev
# activate dev from IDE
```
- Run a command in a specific environment:
```sh
pixi run -e ci-py309-min-deps COMMAND
```
- Spawn a shell with the given env:
```sh
pixi shell -e ci-latest-optional-deps
```
:::

::: {.fragment}
My recommendation: use pixi! 

:::

## Note on the `pixi.lock` file

::: {.callout-important}
If you use pixi, it may happen that the `pixi.lock` file will be updated as you
run commands. 

**Revert** changes to this file before adding files and pushing upstream. 
:::


## Running tests {.smaller}
::: {.panel-tabset}
### Using environments
From inside the root skrub folder, and after activating the environment
```sh
pytest --pyargs skrub
```

### With pixi
Run all tests (you will be prompted to choose an env):
```sh
pixi run test
```
Run tests in a specific env
```sh
pixi run -e dev test
```

:::

## Running tests
Tests are stored in `skrub/tests`, or in a `tests` subfolder. 

It is possible to run specific tests by providing a path. To test `TableVectorizer`: 

```sh
pytest -vsl skrub/tests/test_table_vectorizer.py 
```

`-vsl` prints out more information compared to the default. 

## Working on the documentation {.smaller}
Docs are written in RST and use the
[Sphinx library](https://www.sphinx-doc.org/en/master/index.html) for rendering, 
cross-references and everything else. 

::: {.panel-tabset}
### From environment

Build the doc from the `doc` folder using:
```sh
# Build the full documentation, including examples
make html

# Build documentation without running examples (faster)
make html-noplot

# Clean previously built documentation
make clean
```

### With pixi
From the skrub root folder (where `pyproject.toml` is):
```sh
# Build the full documentation, including examples
pixi run build-doc

# Build documentation without running examples (faster)
pixi run build-doc-quick

# Clean previously built documentation
pixi run clean-doc
```
:::

After rendering the docs, open the `doc/_build/html/index.html` file with a 
browser.

## Writing an example
Full guide: `https://skrub-data.org/stable/tutorial_example.html`


## Opening a PR and contributing upstream {.smaller}
More detail is available on the main website:
`https://skrub-data.org/stable/CONTRIBUTING.html`

Start by creating a branch:
```sh
# fetch latest updates and start from the current head
git fetch upstream
git checkout -b my-branch-name-eg-fix-issue-123
```

Make some changes, then:
```sh
git add ./the/file-i-changed
git commit -m "my message"
git push --set-upstream origin my-branch-name-eg-fix-issue-123
```
At this point, visit the GitHub PR page and open a PR from there:
`https://github.com/skrub-data/skrub/pulls`

## Code formatting and `pre-commit` {.smaller}
Formatting is enforced though pre-commit checks.

::: {.panel-tabset}
### From environment
- Make sure `pre-commit` is installed in the environment and the folder
```sh
pre-commit install
```
- If missing, install it with `pip`
```sh
pip install pre-commit
```
Then, add modified files and run pre-commit on them. 

```sh
git add YOUR_FILE
pre-commit
# if the file has been formatted, add again
git add YOUR_FILE
# commit the changes
git commit -m "MY COMMIT MESSAGE"
# pre-commit will run again automatically
# push the  commit
git push
```


### Pixi
If you're working entirely with pixi:

```sh
git add YOUR_FILE
pixi run lint
# if the file has been formatted, add again
git add YOUR_FILE
# commit the changes
git commit -m "MY COMMIT MESSAGE"
# pre-commit will run again automatically
# push the  commit
git push
```
:::


## Where to find this presentation
::: {.columns}
::: {.column}

Slides
![](images/qr-code.png)

:::
::: {.column}
GitHub project
![](images/qr-code-github-project.png)

:::
::: 